In [1]:
from pandas import read_csv 
import numpy as np

In [15]:
#df = read_csv('international-airline-passengers.csv', usecols=[1])
df = read_csv('international-airline-passengers.csv')


df

,Month,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
5,1949-06,135
6,1949-07,148
7,1949-08,148
8,1949-09,136
9,1949-10,119


In [16]:
values = df.values.astype('float32')
values.shape

ValueError: could not convert string to float: '1960-12'

In [4]:
train_size = int((values.shape[0] * 0.67))
test_size = values.shape[0]  - train_size
train = values[0:train_size]
test = values[train_size:]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range =(0,1))
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [7]:
def create_dataset(data, k):
    dataX, dataY = [],[]
    for i in range(data.shape[0] - k):
        x = data[i:i + k, 0]
        y = data[i + k, 0]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [8]:
look_back = 12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [9]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
trainX.shape

(84, 12, 1)

In [12]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

In [28]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size = 1, validation_data=(testX, testY))
model.summary()

Train on 84 samples, validate on 36 samples
Epoch 1/10
84/84 [==============================] - 1s 7ms/step - loss: 0.0181 - val_loss: 0.1854
Epoch 2/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0138 - val_loss: 0.1779
Epoch 3/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0128 - val_loss: 0.1770
Epoch 4/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0122 - val_loss: 0.1675
Epoch 5/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0114 - val_loss: 0.1689
Epoch 6/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0112 - val_loss: 0.1574
Epoch 7/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0104 - val_loss: 0.1440
Epoch 8/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0098 - val_loss: 0.1351
Epoch 9/10
84/84 [==============================] - 0s 3ms/step - loss: 0.0095 - val_loss: 0.1377
Epoch 10/10
84/84 [==============================] - 0s 3ms/step - loss: 0

In [17]:
y_pred = model.predict(testX)

In [29]:
score = model.evaluate(testX,testY)
score

36/36 [==============================] - 0s 136us/step


0.12700668474038443